####ADVANCED READ OPERATIONS

In [0]:
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.getOrCreate()


In [0]:
%sql
create catalog data_catalog;
create schema data_catalog.data_schema;
create volume data_catalog.data_schema.data_volume;

In [0]:
df=spark.read.csv("/Volumes/data_catalog/data_schema/data_volume/data.csv",header="True",mode="PERMISSIVE")
display(df)

In [0]:
df=spark.read.csv("/Volumes/data_catalog/data_schema/data_volume/data.csv",header=True,mode="dropMalformed")
display(df)

In [0]:
df = spark.read.csv(
    "/Volumes/data_catalog/data_schema/data_volume/data.csv",
    header=True,
    mode="failFast"
)
#fails as there is a corrupted record
display(df)

In [0]:
df = spark.read.csv(
    "/Volumes/data_catalog/data_schema/data_volume/data.csv",
    header=True,
    columnNameOfCorruptRecord="_corrupt_record",
    mode="permissive"
)

if "_corrupt_record" in df.columns:
    corrupt_df = df.filter(df["_corrupt_record"].isNotNull())
    corrupt_df.write.mode("overwrite").text(
        "/Volumes/data_catalog/data_schema/data_volume/corrupte"
    )
else:
    print("No corrupt records")

display(df)

In [0]:
df=spark.read.csv("/Volumes/data_catalog/data_schema/data_volume/data.csv",header=True,quote='"',comment="#",nullValue=" ",nanValue="NAN",ignoreLeadingWhiteSpace=True,ignoreTrailingWhiteSpace=True)# NaN is also treated as null
display(df)
 # Treat text inside quotes as one value
 #in order to remove the ' before and after the text 
from pyspark.sql.functions import *

columns=df.columns
for column in columns:
  df=df.withColumn(column,regexp_replace(col(column),"'",""))
display(df)
#withcolumn- adds or replace a column to a DataFrame. columnname,columnvalue


In [0]:
df_orc=spark.read.format("orc").option("compression","zlib").option("header","true").option("sep",",").load("/Volumes/telecom_catalog_assign/landing_zone/ingestion_volume/cus_orc_out").toDF("id","name","age","city","plan")
display(df_orc)


In [0]:
df_par=spark.read.format("parquet").option("compression","gzip").option("header","true").option("sep",",").load("/Volumes/telecom_catalog_assign/landing_zone/ingestion_volume/cus_orc_parquet_out").toDF("id","name","age","city","plan")
display(df_par)

In [0]:
df_delta=spark.read.table("telecom_catalog_assign.landing_zone.ingest_c_table")
display(df_delta)